In [24]:
# from nltk.corpus.reader import documents
# from langchain.chains.question_answering.map_reduce_prompt import messages
# !pip install python-dotenv
# !pip install -U openai langchain langchain-openai
!pip install -U langchain-ollama
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu
!pip install chromadb
!pip install langchain-teddynote

ModuleNotFoundError: No module named 'pydantic'

In [2]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging
# import openai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.vectorstores import Chroma
import time

In [3]:
load_dotenv()
logging.langsmith("mammothLangchain", set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
mammothLangchain


In [21]:
def pdf_loader(file_path):
    loader = PyPDFLoader(file_path)
    return loader

In [23]:
# 벡터 스토어를 새로 만들 경우, 저장하기 까지 시간이 걸려서 메모리에 저장
def create_vectorstore(file_id, file_texts, embeddings, vectorstore_path):

    index = VectorstoreIndexCreator(
        vectorstore_cls=Chroma,
        embedding=embeddings,
        vectorstore_kwargs={
            "persist_directory":vectorstore_path,
            # "collection_name":get_collection_name(file_name),
        }
        ).from_loaders([file_texts])

    return index

In [6]:
def load_vectorstore(embeddings, vectorstore_path):
    index = VectorStoreIndexWrapper(vectorstore=vectorstore_path)
    return index

In [7]:
def get_vectorstore(file_id, file, embeddings, vectorstore_path, db_already_exist):
    if db_already_exist:
        index = load_vectorstore(embeddings, vectorstore_path)
    else:
        index = create_vectorstore(file_id, file, embeddings, vectorstore_path)
    return index

In [8]:
def set_model(model):
    chat = ChatOllama(
        model=model,
        temperature=0.1,
        num_predict=512,
        top_p=1,
        mirostat_tau=0.4,
        # CPU 코어수
        num_thread=None,
    )
    return chat

In [16]:
def set_chat():
    file_path = "C:/Users/user/Documents/GitLab/Test_Dawoo/company_ask/doc/test/소나기 - 황순원.pdf"
    vectorstore_path = "./vectorstore2"
    model = 'llama3.2-bllossom-kor-3B:latest'
    embeddings = OllamaEmbeddings(model="mxbai-embed-large")
    #embeddings = HuggingFaceEmbeddings()

    # env_loader()
    file_texts = pdf_loader(file_path)
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    # show_pdf(file)
    index = get_vectorstore(
        file_id=file_name,
        file=file_texts,
        embeddings = embeddings,
        vectorstore_path=vectorstore_path,
        db_already_exist=False,
    )
    chat = set_model(model=model)

    return chat, index

In [10]:
def chat_input(index, user_input, chat):
    return index.query( user_input, llm=chat, )

In [25]:
chat, index = set_chat()

while True:
    user_input = input("you: ")
    print(f"you: {user_input}")

    if user_input == "exit":
        break
    elif user_input.replace(" ", "") == "":
        print("chat: 질문이 없습니다. 정확한 질문을 해주시면 답변해 드리겠습니다.")
    else:
        chat_text = chat_input(
            chat=chat,
            index=index,
            user_input=user_input,
        )
        print(f"chat: {chat_text}")
        time.sleep(0.5)

you: 원하는 결과를 빠르게 도출.', '사용자 관리단말기 관리생체정보 관리라이선스 관리게시판 관리통합 검색엔진권한 관리서비스 관리SSODevOps 관리\nDevOps', 'MammothData DevOpsService Mesh\nAMQP\nIngress\nReact\nReact\nMammoth 검색엔진\nAPI Gateway', '• 세계 보안업체들과 협업을 통해 생체인증(안면인식, 지문, 정맥, 카드, 모바일) 출입 통제 보안 미들웨어 서비스 제공• 생체인증을 통해 각 보안업체와 체계적인 출입 보안 강화\n기존 운영 환경에서추가 생체정보 등록 관리기존 사용자 정보 연동 및생체정보, 단말기 등록 관리방문자 예약 및
이 내용에 한줄로 제목을 지어줘.
chat: "DevOps 및 생체인증 서비스 개발: 통합 관리와 보안 강화"


KeyboardInterrupt: Interrupted by user